<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# Ejercicio de clasificación con Random Forest

Ejemplo de clasificación utilizando random forest para la clasificación de drogadas que debería tomar un pasiente según su historial clínico<br>

v1.1

In [ ]:
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

In [ ]:
if os.access('drug200.csv', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/drug200.csv > drug200.csv
    else:
        !wget drug200.csv https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/drug200.csv

### `drug200.csv`:
El dataset **`drug200.csv`** contiene diferentes tipos de drogas que se le dan a pacientes relativo a su historial clínico. El objetivo es dado un nuevo paciente clasificarlo y determinar que droga es la más apropiada para el.<br> [Dataset source](https://www.kaggle.com/jeevanrh/drug200csv)

- **Age** --> edad, ejemplo 25
- **Sex** --> género, ejemplo F(femenino), M(masculino)
- **BP (Blood Pressure)** --> presión arterial, ejemplo HIGH(alta)
- **Cholesterol** --> colesterol, ejemplo normal (NORMAL)
- **Na / k** --> concentración de sodio/potasio en sangre, ejemplo 7.8
- **Drug** --> droga suministrada, ejemplo drugC

# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

In [ ]:
df = pd.read_csv("drug200.csv")
des = df.describe()
des.loc['Nan'] = df.isna().sum()
des.loc['%Nan'] = (df.isna().mean())*100
des

In [ ]:
df.head()

In [ ]:
print('Cantidad de datos en observacion:', df.shape[0])

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
# Exploramos que tan balanceado está el dataset,
# en cuantos casos se suministró cada droga
df['Drug'].value_counts()

In [ ]:
ax = sns.countplot(data=df, x="Drug")

Se puede observar que en la mayoría de los casos se suministra la drogaY oa la drogaX, es muy probable que el modelo siga esta tendencia

#### Transformar variables categóricas texto a clases numeradas

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

df_cod = df.copy()

In [ ]:
df.head()

In [ ]:
def one_hot_encoding(df, column):
    df_copy = df.copy()
    # LabelEncoder
    le = LabelEncoder()
    label_encoding = le.fit_transform(df_copy[column])
    # OneHotEncoder
    onehot_encoder = OneHotEncoder(sparse=False)
    one_hot_encoding = onehot_encoder.fit_transform(label_encoding.reshape(-1, 1))
    # Crear las columnas con el resultado del encoder
    one_hot_encoding_df = pd.DataFrame(one_hot_encoding, columns=le.classes_, dtype=int)
    # Agregar sufijo
    one_hot_encoding_df = one_hot_encoding_df.add_prefix(column+'_')
    # Unir nuevas columnas al dataset
    df_copy = df_copy.join(one_hot_encoding_df)
    # Eleminar vieja columna del dataset
    df_copy = df_copy.drop([column], axis=1)
    return df_copy, label_encoding, one_hot_encoding


In [ ]:
df_cod, le_sex, ohe_sex = one_hot_encoding(df_cod, 'Sex')
df_cod, le_bp, ohe_bp = one_hot_encoding(df_cod, 'BP')
df_cod, le_cholest, ohe_cholest = one_hot_encoding(df_cod, 'Cholesterol')

In [ ]:
df_cod.head()

# Entrenar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

El primer paso es obtener los datos que serán la entrada del sistema (X) y los datos que serán la salida del modelo estimador (y)

In [ ]:
X = df_cod.drop('Drug', axis=1).values
y = df_cod['Drug'].values

Siguiente paso es dividir el dataset en entrenamiento (train) y evaluación (test). Utilizaremos el criterio 70%30%

In [ ]:
from sklearn.model_selection import train_test_split
# Fijamos un "random_state" constante para que siempre el dataset se parta de la misma forma
# para poder repetir los ensayos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Creamos el modelo base
class RandomBaseModel():
    def __init__(self):
        self.classes_ = [0, 1]
    def fit(self,X, y):
        self.classes_ = np.unique(y)
        return None

    def predict(self,X):
        rand = np.random.randint(0, len(self.classes_), size=X.shape[0])
        rand_clases = [self.classes_[x] for x in rand]
        return np.asarray(rand_clases)

In [ ]:
# Obtener la salida según el modelo base
random_model = RandomBaseModel()
random_model.fit(X_train, y_train)
y_hat_base = random_model.predict(X_test)
random_model.classes_

#### Crear un modelo de clasificación con random forest
Parámetros
- n_estimators --> Cantidad de árboles (por defecto 100)
- max_depth --> Máxima profundiad del árbol, por defecto sin límite
- min_samples_leaf --> Cantidad mínima de muestras en un nodo para considerarse leaft (defecto 1)
- criterion --> Criterio para crear los nodos (entropy o gini)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=4, max_depth=5, criterion='entropy', random_state = 0)
clf.fit(X_train, y_train)
y_hat = clf.predict(X_test)

# Validar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline5.png" width="1000" align="middle">

In [ ]:
# Calcular la exactitud (accuracy)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_hat_base, normalize=True)

In [ ]:
# Calcular la exactitud (accuracy)
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_hat, normalize=True)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, y_hat)
cmd = ConfusionMatrixDisplay(cm, display_labels=clf.classes_)
cmd.plot(cmap=plt.cm.Blues)
plt.show()

# Utilizar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline6.png" width="1000" align="middle">

In [ ]:
from sklearn import tree

# Tomo el primer árbol de decisión del bosque para dibujar
estimator = clf.estimators_[0]

# Obtengo los nombres de las coulmnas utilizadas y las clases posibles
feature_names = df_cod.drop('Drug', axis=1).columns
class_names = df_cod['Drug'].unique().tolist()

fig = plt.figure(figsize=(60,40))
ax = fig.add_subplot()
tree.plot_tree(estimator,
               feature_names = feature_names, 
               class_names=class_names,
               filled = True, ax=ax);
plt.show()
filename = 'drugtree.png'
fig.savefig(filename)

# Conclusión
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline7.png" width="1000" align="middle">

En este ejemplo no fue muy dificil conseguir un buen resultado ya que no era un dataset complejo. La misma estrategía de trabajo puede aplicarse para otros datasets

#### Exportar

In [ ]:
import pickle
# Salvar el modelo
pickle.dump(clf, open('randomforest_model.pkl', 'wb'))
# Salvar los codificadores
pickle.dump(le_sex, open('le_sex.pkl', 'wb'))
pickle.dump(ohe_sex, open('ohe_sex.pkl', 'wb'))
pickle.dump(le_bp, open('le_bp.pkl', 'wb'))
pickle.dump(ohe_bp, open('ohe_bp.pkl', 'wb'))
pickle.dump(le_cholest, open('le_cholest.pkl', 'wb'))
pickle.dump(ohe_cholest, open('ohe_cholest.pkl', 'wb'))